In [75]:
import pandas as pd
import numpy as np



In [76]:
train = pd.read_csv('./Data/train_cleaned.csv')
test = pd.read_csv('./Data/test_cleaned.csv')
labels = pd.read_csv('PumpItUp_Labels.csv')
train = train.merge(labels, on='id')
target=train.pop('status_group')

train['train'] = 1
test['train'] = 0

train.drop('Unnamed: 0',inplace=True,axis=1)
test.drop('Unnamed: 0',inplace=True,axis=1)

total = pd.concat([train,test])

In [77]:
train.columns

Index(['id', 'amount_tsh', 'funder', 'gps_height', 'installer', 'longitude',
       'latitude', 'basin', 'subvillage', 'region', 'region_code',
       'district_code', 'lga', 'ward', 'population', 'public_meeting',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'management', 'payment_type', 'water_quality',
       'quantity', 'source', 'waterpoint_type', 'operation_years', 'train'],
      dtype='object')

In [78]:
target.value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [79]:
proportion = (target.value_counts()/target.value_counts().sum())
proportion

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

In [80]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74250 entries, 0 to 14849
Data columns (total 29 columns):
id                   74250 non-null int64
amount_tsh           74250 non-null float64
funder               74250 non-null object
gps_height           74250 non-null float64
installer            74250 non-null object
longitude            74250 non-null float64
latitude             74250 non-null float64
basin                74250 non-null object
subvillage           73780 non-null object
region               74250 non-null object
region_code          74250 non-null int64
district_code        74250 non-null int64
lga                  74250 non-null object
ward                 74250 non-null object
population           74250 non-null float64
public_meeting       70095 non-null object
scheme_management    69404 non-null object
scheme_name          74250 non-null object
permit               70457 non-null object
construction_year    74250 non-null float64
extraction_type      74250 n

In [81]:
total.columns

Index(['id', 'amount_tsh', 'funder', 'gps_height', 'installer', 'longitude',
       'latitude', 'basin', 'subvillage', 'region', 'region_code',
       'district_code', 'lga', 'ward', 'population', 'public_meeting',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'management', 'payment_type', 'water_quality',
       'quantity', 'source', 'waterpoint_type', 'operation_years', 'train'],
      dtype='object')

In [82]:
total = total.drop(['funder','scheme_name','subvillage','region','region_code','district_code','lga','ward',
              'public_meeting','water_quality', 'construction_year'], axis = 1)

In [84]:
cols = ['installer','scheme_management','permit']
for col in cols:
    if total[col].value_counts().index[0]==0:
        total[col] = total[col].fillna(total[col].value_counts().index[1])
    else:
        total[col] = total[col].fillna(total[col].value_counts().index[0])

In [85]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 29 columns):
id                   59400 non-null int64
amount_tsh           59400 non-null float64
funder               59400 non-null object
gps_height           59400 non-null float64
installer            59400 non-null object
longitude            59400 non-null float64
latitude             59400 non-null float64
basin                59400 non-null object
subvillage           59029 non-null object
region               59400 non-null object
region_code          59400 non-null int64
district_code        59400 non-null int64
lga                  59400 non-null object
ward                 59400 non-null object
population           59400 non-null float64
public_meeting       56066 non-null object
scheme_management    55523 non-null object
scheme_name          59400 non-null object
permit               56344 non-null object
construction_year    59400 non-null float64
extraction_type      59400 n

In [86]:
#divide data again into train and test

train = total.iloc[:59400,:]
test = total.iloc[59400:,:]

In [87]:
#installer dataframe & dummified target
train_installer_target = pd.concat([train['installer'],target], axis=1)
target_dummies = pd.get_dummies(train_installer_target['status_group'])
train_installer_target = train_installer_target.drop(['status_group'], axis = 1)
train_installer_target = pd.concat([train_installer_target,target_dummies], axis=1)
train_installer_target.head(2)

,installer,functional,functional needs repair,non functional
0,DWE,0,0,1
1,DWE,0,0,1


In [88]:
train_installer_target_prop = pd.concat([train_installer_target.groupby('installer').mean(),train_installer_target.groupby('installer').count()], axis = 1).iloc[:,:-2]
train_installer_target_prop.columns = ['functional','functional needs repair','non functional','count']
train_installer_target_prop.sort_values(by='count',ascending=False).head()

,functional,functional needs repair,non functional,count
installer,,,,
Others,0.561979,0.061091,0.376930,33802
DWE,0.542064,0.093208,0.364728,17402
Government,0.293151,0.140274,0.566575,1825
RWE,0.252073,0.113599,0.634328,1206
Commu,0.683019,0.030189,0.286792,1060


In [89]:
train_installer_target_prop['category'] = np.zeros((train_installer_target_prop.shape[0],1))
train_installer_target_prop['category'].loc[train_installer_target_prop['functional'] > 0.54] = 'functional'
train_installer_target_prop['category'].loc[train_installer_target_prop['non functional'] > 0.38] = 'non functional'
train_installer_target_prop['category'].loc[train_installer_target_prop['functional needs repair'] > 0.05] = 'functional needs repair'

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [90]:
train['installer_category'] = train.installer.map(train_installer_target_prop.category)
test['installer_category'] = test.installer.map(train_installer_target_prop.category)
test['installer_category'] = test['installer_category'].fillna(test['installer_category'].value_counts().index[0])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [91]:
total = pd.concat([train,test])

In [92]:
total = total.drop(['installer'], axis=1)

In [93]:
total.describe(include = ['O'])

,basin,scheme_management,extraction_type,management,payment_type,quantity,source,waterpoint_type,installer_category
count,74250,74250,74250,74250,74250,74250,74250,74250,74250
unique,9,12,18,12,7,5,10,7,3
top,Lake Victoria,VWC,gravity,vwc,never pay,enough,spring,communal standpipe,functional needs repair
freq,12871,50763,33263,50624,31712,41522,21216,35628,71068


In [94]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74250 entries, 0 to 14849
Data columns (total 18 columns):
id                    74250 non-null int64
amount_tsh            74250 non-null float64
gps_height            74250 non-null float64
longitude             74250 non-null float64
latitude              74250 non-null float64
basin                 74250 non-null object
population            74250 non-null float64
scheme_management     74250 non-null object
permit                74250 non-null bool
extraction_type       74250 non-null object
management            74250 non-null object
payment_type          74250 non-null object
quantity              74250 non-null object
source                74250 non-null object
waterpoint_type       74250 non-null object
operation_years       74250 non-null int64
train                 74250 non-null int64
installer_category    74250 non-null object
dtypes: bool(1), float64(5), int64(3), object(9)
memory usage: 10.3+ MB


In [95]:
total_dummies = pd.get_dummies(total)

In [96]:
total_dummies.head()

,id,amount_tsh,gps_height,longitude,latitude,population,permit,operation_years,train,basin_Internal,...,waterpoint_type_cattle trough,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,installer_category_functional,installer_category_functional needs repair,installer_category_non functional
0,21669,777.777778,1715.0,31.366025,-7.972010,1.0,True,37,1,0,...,0,1,0,0,0,0,0,0,1,0
1,60108,1352.941176,1906.0,31.431333,-7.850336,1.0,True,35,1,0,...,0,1,0,0,0,0,0,0,1,0
2,44820,777.777778,1741.0,31.368317,-7.971081,1.0,True,37,1,0,...,0,1,0,0,0,0,0,0,1,0
3,53307,777.777778,1730.0,31.368730,-7.972788,1.0,True,37,1,0,...,0,1,0,0,0,0,0,0,1,0
4,5701,777.777778,1731.0,31.365461,-7.970985,1.0,True,37,1,0,...,0,1,0,0,0,0,0,0,1,0


In [97]:
total_dummies.shape

(74250, 92)

In [98]:
train = total_dummies.iloc[:59400,:]
test = total_dummies.iloc[59400:,:]

In [99]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [100]:
classifier = RandomForestClassifier(n_estimators=500, random_state = 8)


In [101]:
score = cross_val_score(classifier, train, np.ravel(target), scoring = 'accuracy',cv=5).mean()
print("Score: {}".format(score))

Score: 0.6949494061101004
